<a href="https://colab.research.google.com/github/bf319/Scaling_MPNNs/blob/main/final_version_1_padding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -q git+https://github.com/deepmind/dm-haiku
%pip install -q jraph
%pip install -q git+https://github.com/deepmind/jaxline
%pip install -q ogb
%pip install -q dgl
%pip install -q optax
%pip install -q metis

In [3]:
from ogb.nodeproppred import Evaluator
from ogb.nodeproppred import DglNodePropPredDataset

dataset = DglNodePropPredDataset(name = "ogbn-proteins")
split_idx = dataset.get_idx_split()
evaluator = Evaluator(name = 'ogbn-proteins')

Using backend: pytorch


In [4]:
import jax.numpy as jnp

# There is only one graph in Node Property Prediction datasets
ogbn_proteins_main_graph, ogbn_proteins_main_labels = dataset[0]

'''
  OGBN-Proteins
    #Nodes = 132,534
    #Edges = 39,561,252
    #Tasks = 112
    #Split Type = Species
    #Task Type = Binary classification
    #Metric = ROC-AUC

    Task:
      The task is to predict the presence of protein functions in a multi-label binary classification setup,
      where there are 112 kinds of labels to predict in total. 
      The performance is measured by the average of ROC-AUC scores across the 112 tasks.

    #Others:
      **undirected**
      **weighted**
      **typed (according to species)**

  (1) Nodes represent proteins
    (1.1) The proteins come from 8 species
      len(set(graph.ndata['species'].reshape(-1).tolist())) == 8
    (1.2) Each node has one feature associated with it (its species)
      graph.ndata['species'].shape == (#nodes, 1)
  
  (2) Edges indicate different types of biologically meaningful associations between proteins
    (2.1) All edges come with 8-dimensional features
      graph.edata['feat'].shape == (2 * #edges, 8)

'''
# Get split labels
train_label = dataset.labels[split_idx['train']]  # (86619, 112) -- binary values (presence of protein functions)
valid_label = dataset.labels[split_idx['valid']]  # (21236, 112) -- binary values (presence of protein functions)
test_label = dataset.labels[split_idx['test']]    # (24679, 112) -- binary values (presence of protein functions)

# Create masks
train_mask = jnp.zeros((ogbn_proteins_main_graph.num_nodes(), 1)).at[jnp.array(split_idx['train'])].set(1)
valid_mask = jnp.zeros((ogbn_proteins_main_graph.num_nodes(), 1)).at[jnp.array(split_idx['valid'])].set(1)
test_mask = jnp.zeros((ogbn_proteins_main_graph.num_nodes(), 1)).at[jnp.array(split_idx['test'])].set(1)

In [5]:
import jraph

# From https://colab.research.google.com/github/deepmind/educational/blob/master/colabs/summer_schools/intro_to_graph_nets_tutorial_with_jraph.ipynb#scrollTo=7vEmAsr5bKN8
def _nearest_bigger_power_of_two(x: int) -> int:
  """Computes the nearest power of two greater than x for padding."""
  y = 2
  while y < x:
    y *= 2
  return y

def pad_graph_to_nearest_power_of_two(
    graphs_tuple: jraph.GraphsTuple) -> jraph.GraphsTuple:
  """
  Pads a batched `GraphsTuple` to the nearest power of two.
  For example, if a `GraphsTuple` has 7 nodes, 5 edges and 3 graphs, this method
  would pad the `GraphsTuple` nodes and edges:
    7 nodes --> 8 nodes (2^3)
    5 edges --> 8 edges (2^3)
  And since padding is accomplished using `jraph.pad_with_graphs`, an extra
  graph and node is added:
    8 nodes --> 9 nodes
    3 graphs --> 4 graphs
  Args:
    graphs_tuple: a batched `GraphsTuple` (can be batch size 1).
  Returns:
    A graphs_tuple batched to the nearest power of two.
  """
  # Add 1 since we need at least one padding node for pad_with_graphs.
  pad_nodes_to = _nearest_bigger_power_of_two(jnp.sum(graphs_tuple.n_node)) + 1
  pad_edges_to = _nearest_bigger_power_of_two(jnp.sum(graphs_tuple.n_edge))
  # Add 1 since we need at least one padding graph for pad_with_graphs.
  # We do not pad to nearest power of two because the batch size is fixed.
  pad_graphs_to = graphs_tuple.n_node.shape[0] + 1
  return jraph.pad_with_graphs(graphs_tuple, pad_nodes_to, pad_edges_to,
                               pad_graphs_to)

In [6]:
import numpy as np
import torch
import jraph

def dgl_graph_to_jraph(node_ids):
  # First add back the node and edge features
  dgl_graph_with_features = dgl.node_subgraph(ogbn_proteins_main_graph, node_ids)

  node_features = jnp.array(dgl_graph_with_features.ndata['species'])
  senders = jnp.array(dgl_graph_with_features.edges()[0])
  receivers = jnp.array(dgl_graph_with_features.edges()[1])

  # Edges -- here we should include the 8-dimensional edge features
  edges = jnp.array(dgl_graph_with_features.edata['feat'])

  n_node = jnp.array([dgl_graph_with_features.num_nodes()])
  n_edge = jnp.array([dgl_graph_with_features.num_edges()])

  return pad_graph_to_nearest_power_of_two(jraph.GraphsTuple(
            nodes = node_features.astype(np.float32),
            senders = senders.astype(np.int32), 
            receivers = receivers.astype(np.int32),
            edges = edges.astype(np.float32),  
            n_node = n_node, 
            n_edge = n_edge,
            globals = None  # No global features
          ))
  
def get_labels_for_subgraph(node_ids):
  return jnp.array(ogbn_proteins_main_labels.index_select(0, node_ids))

In [7]:
import dgl

'''
  Generate graph partition using metis, with balanced number of edges in each partition.
  Note: 
    The subgraphs do not contain the node/edge data in the input graph (https://docs.dgl.ai/generated/dgl.metis_partition.html)
'''
num_partitions = 100

# reshuffle == False, so my understandinng is that
# partition.ndata['_ID'] uses the same ids for the initial nodes
# therefore, the initial train / valid / test masks should still work.
dgl_graph_metis_partition = dgl.metis_partition(ogbn_proteins_main_graph, num_partitions, balance_edges = True)

Convert a graph into a bidirected graph: 3.159 seconds
Construct multi-constraint weights: 0.022 seconds
Metis partitioning: 31.546 seconds
Split the graph: 1.728 seconds
Construct subgraphs: 0.027 seconds


In [8]:
# Convert graphs to Jraph GraphsTuple
processed_graphs = {}

for idx in range(num_partitions):
  node_ids = dgl_graph_metis_partition[idx].ndata['_ID']

  graph = dgl_graph_to_jraph(node_ids)
  labels = get_labels_for_subgraph(node_ids)

  processed_graphs[f'partition_{idx}'] = {
      'graph': graph, 
      'labels': labels,
      'train_mask': train_mask.at[jnp.array(node_ids)].get(),
      'valid_mask': valid_mask.at[jnp.array(node_ids)].get(),
      'test_mask': test_mask.at[jnp.array(node_ids)].get()
      }

In [9]:
import haiku as hk
import jax
import optax

from typing import Sequence

# See https://github.com/YuxuanXie/mcl/blob/5f7ee92e2a6bc89736263873a4ba9c14d1a676ff/glassy_dynamics/train_using_jax.py for alternative to using GraphMapFeatures
# From https://github.com/YuxuanXie/mcl/blob/5f7ee92e2a6bc89736263873a4ba9c14d1a676ff/glassy_dynamics/train_using_jax.py

hidden_dimension = 64
num_message_passing_steps = 3

@jraph.concatenated_args
def node_update_fn(feats: jnp.ndarray) -> jnp.ndarray:
  """Node update function for graph net."""
  net = hk.Sequential([hk.nets.MLP(output_sizes = [hidden_dimension, hidden_dimension]), jax.nn.relu, hk.LayerNorm(axis = -1, create_scale = False, create_offset = False)])
  return net(feats)

@jraph.concatenated_args
def edge_update_fn(feats: jnp.ndarray) -> jnp.ndarray:
  """Edge update function for graph net."""
  net = hk.Sequential([hk.nets.MLP(output_sizes = [hidden_dimension, hidden_dimension]), jax.nn.relu, hk.LayerNorm(axis = -1, create_scale = False, create_offset = False)])
  return net(feats)

def node_decoder_fn(feats: jnp.ndarray) -> jnp.ndarray:
  ## TODO: Check if this is correct
  net = hk.Sequential([hk.nets.MLP(output_sizes = [112], activate_final = False), jax.nn.sigmoid])
  return net(feats)

def network_definition(graph):
  """Defines a graph neural network.
  Args:
    graph: Graphstuple the network processes.
  Returns:
    Decoded nodes.
  """

  embedder = jraph.GraphMapFeatures(
      embed_node_fn=hk.Linear(hidden_dimension),
      embed_edge_fn=hk.Linear(hidden_dimension),
      )
  graph = embedder(graph)

  gn = jraph.InteractionNetwork(
      update_node_fn=node_update_fn,
      update_edge_fn=edge_update_fn,
      include_sent_messages_in_node_update=True
      )

  for _ in range(num_message_passing_steps):
    graph = gn(graph)

  decoder = jraph.GraphMapFeatures(embed_node_fn = node_decoder_fn)
  
  processed_graph = decoder(graph)
  return processed_graph.nodes

In [16]:
import functools
import haiku as hk

from random import randint

# Try to follow this tutorial https://github.com/YuxuanXie/mcl/blob/5f7ee92e2a6bc89736263873a4ba9c14d1a676ff/glassy_dynamics/train_using_jax.py
def compute_loss(params, graph, label, mask):
  # Question: would the net need to be passed from the train function?
  # I don't think so because this is just a function and we pass the parameters
  # below, but I just wanted to confirm.
  net = hk.without_apply_rng(hk.transform(network_definition))

  predictions = net.apply(params, graph)
  predictions_after_masked_nodes_are_removed = predictions.at[jraph.get_node_padding_mask(graph)].get() # Shape == label.shape
  
  # Question: Is node masking implemented properly?
  loss = optax.sigmoid_binary_cross_entropy(predictions_after_masked_nodes_are_removed, label)
  loss = loss * mask
  loss = loss.mean()
  return loss

def train(num_training_steps):
  # Transform the function (MPNN) into a pure function (with no side effects) so that it can be used with jax
  net = hk.without_apply_rng(hk.transform(network_definition))

  params = net.init(jax.random.PRNGKey(42), processed_graphs['partition_0']['graph'])

  opt_init, opt_update = optax.adam(learning_rate = 0.1)  # This learning rate could be a bit low, try 1e-1 for full graph descent?  
  opt_state = opt_init(params)

  def update(params, opt_state, graph, targets, mask):
    # Compute the gradients on the given minibatch (individually on each device).
    loss, grads = jax.value_and_grad(compute_loss)(params, graph, targets, mask)

    updates, opt_state = opt_update(grads, opt_state)
    return optax.apply_updates(params, updates), opt_state, loss

  # Train
  for idx in range(num_training_steps):
    random_partition_idx = randint(0, num_partitions - 1)
    random_partition = processed_graphs[f'partition_{random_partition_idx}']

    graph = random_partition['graph']
    labels = random_partition['labels']
    mask = random_partition['train_mask']

    params, opt_state, loss = update(
        params, 
        opt_state, 
        graph, 
        labels,
        mask
        )
    print('Loss training:', loss)

    # TODO: Write params and opt_state

  return params

def evaluate(params, num_graphs_eval):
  # Evaluate
  accumulated_loss = 0.0
  accumulated_roc = 0

  for idx in range(num_graphs_eval):
    random_partition_idx = randint(0, num_partitions - 1)
    random_partition = processed_graphs[f'partition_{random_partition_idx}']

    graph = random_partition['graph']
    labels = random_partition['labels']
    mask = random_partition['test_mask']

    (loss, roc) = evaluate_graph(params, graph, labels, mask)

    accumulated_loss += loss
    accumulated_roc += roc

  print(f'Average test loss: {accumulated_loss / num_graphs_eval} | Average ROC: {accumulated_roc / num_graphs_eval}')

def evaluate_graph(params, graph, label, mask):
  compute_loss_fn = functools.partial(compute_loss)

  net = hk.without_apply_rng(hk.transform(network_definition))
  decoded_nodes = net.apply(params, graph) # Shape == label.shape
  
  ## Question: Predictions should be rounded here right?
  predictions = jax.lax.round(decoded_nodes)
  predictions_after_masked_nodes_are_removed = predictions.at[jraph.get_node_padding_mask(graph)].get() # Shape == label.shape

  loss = compute_loss_fn(params, graph, label, mask)
  roc = evaluator.eval({"y_true": np.array(label), "y_pred": np.array(predictions_after_masked_nodes_are_removed)})['rocauc']

  print(f'Test loss: {loss} | ROC: {roc}')
  return (loss, roc)

final_params = train(num_training_steps = 10)
evaluate(final_params, 5)

Test loss: 0.05983279272913933 | ROC: 0.5
Test loss: 0.24354977905750275 | ROC: 0.5
Test loss: 0.1785455197095871 | ROC: 0.5
Test loss: 0.08673280477523804 | ROC: 0.5
Test loss: 0.0038074059411883354 | ROC: 0.5
Average test loss: 0.11449366807937622 | Average ROC: 0.5


In [21]:
def evaluate_on_full_sets(params):
  net = hk.without_apply_rng(hk.transform(network_definition))

  final_predictions = {}

  for i in range(num_partitions):
    node_ids = dgl_graph_metis_partition[i].ndata['_ID']

    partition = processed_graphs[f'partition_{i}']
    decoded_nodes = net.apply(params, partition['graph'])
    predictions = jax.lax.round(decoded_nodes)
    predictions_after_masked_nodes_are_removed = predictions.at[jraph.get_node_padding_mask(partition['graph'])].get() # Shape == label.shape
    predictions_after_masked_nodes_are_removed = np.array(predictions_after_masked_nodes_are_removed)

    for index, node_id in enumerate(node_ids):
      final_predictions[node_id] = predictions_after_masked_nodes_are_removed[index]

    if (i + 1) % 10 == 0:
      print(f'Evaluated {i + 1} / {num_partitions} subgraphs...')

  # Sort the final predictions based on the node ids
  predictions_in_order = dict(sorted(final_predictions.items()))

  # Convert the values to a list to be able to slice based on the ids of the 
  # nodes in the test set
  predictions_in_order = list(predictions_in_order.values())

  final_roc_train = evaluator.eval({
      "y_true": np.array(train_label), 
      "y_pred": np.array([predictions_in_order[x] for x in split_idx['train']])
      })['rocauc']

  final_roc_valid = evaluator.eval({
      "y_true": np.array(valid_label), 
      "y_pred": np.array([predictions_in_order[x] for x in split_idx['valid']])
      })['rocauc']

  final_roc_test = evaluator.eval({
      "y_true": np.array(test_label), 
      "y_pred": np.array([predictions_in_order[x] for x in split_idx['test']])
      })['rocauc']

  print()
  print(f'Final ROC on the train set {final_roc_train}')
  print(f'Final ROC on the validation set {final_roc_valid}')
  print(f'Final ROC on the test set {final_roc_test}')

evaluate_on_full_sets(final_params)

Evaluated 10 / 100 subgraphs...
Evaluated 20 / 100 subgraphs...
Evaluated 30 / 100 subgraphs...
Evaluated 40 / 100 subgraphs...
Evaluated 50 / 100 subgraphs...
Evaluated 60 / 100 subgraphs...
Evaluated 70 / 100 subgraphs...
Evaluated 80 / 100 subgraphs...
Evaluated 90 / 100 subgraphs...
Evaluated 100 / 100 subgraphs...

Final ROC on the train set 0.5
Final ROC on the validation set 0.5
Final ROC on the test set 0.5
